# Pandas alapok

In [36]:
import pandas as pd

- A Pandas egy táblázatkezelő python alatt.
- Táblázatos formában tárolhatunk vele adatokat, ahol az oszlopoknak neve van a sorok pedig indexelve vannak.
- A táblázatok típusa a DataFrame.

In [37]:
data = {
    'Country': ['Hungary','Colombia','Chile','Equador','Nigeria'],
    'Rank':[121,40,100,130,11]
    }
df = pd.DataFrame(data, index=['a', 'b', 'c', 'd', 'e'])
df

,Country,Rank
a,Hungary,121
b,Colombia,40
c,Chile,100
d,Equador,130
e,Nigeria,11


### Adatok lekérdezése

- Oszlopokat a nevük segítségével

In [38]:
df['Country']

a     Hungary
b    Colombia
c       Chile
d     Equador
e     Nigeria
Name: Country, dtype: object

- Sorokat szintén a nevükkel, viszont a loc objektumváltozó alatt

In [39]:
df.loc[['a']]

,Country,Rank
a,Hungary,121


- Cellát az oszlop és a sor nevével

In [40]:
df['Country']['a']

'Hungary'

- Vagy az iloc objektumváltozó alatt a sorszámával tudunk lekérdezni

In [41]:
df.iloc[0][0]

C:\Users\rtornai\AppData\Local\Temp\ipykernel_11284\3629555008.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[0][0]


'Hungary'

### Adatok szűrése

- Az adatokat egy if feltételhez hasonló módon tudjuk szűrni

In [42]:
df[df["Rank"] < 100 ]

,Country,Rank
b,Colombia,40
e,Nigeria,11


- vagy akár SQL parancs szerűen

In [43]:
df.query("Rank < 100")

,Country,Rank
b,Colombia,40
e,Nigeria,11


### Adatok módosítása

- Cell értékének változtatása .at['sorindex', 'oszlopindex']

In [44]:
df.at['a', 'Country'] = "Magyarország"
df

,Country,Rank
a,Magyarország,121
b,Colombia,40
c,Chile,100
d,Equador,130
e,Nigeria,11


- Sor hozzáadása dictionaryval

In [45]:
df = df._append({'Country': 'Románia', 'Rank': '122'}, ignore_index=True)
df

,Country,Rank
0,Magyarország,121
1,Colombia,40
2,Chile,100
3,Equador,130
4,Nigeria,11
5,Románia,122


- Sor törlése az index segítségével

In [46]:
df = df.drop([1])
df

,Country,Rank
0,Magyarország,121
2,Chile,100
3,Equador,130
4,Nigeria,11
5,Románia,122


# Pandas és a dataclass-ok kapcsolata

In [47]:
from dataclasses import dataclass, field


@dataclass
class Person:
    id: str = field(hash=True)
    name: str = field(repr=True, compare=False)
    age: int = field(repr=True, compare=False)
    male: bool = field(default=True, repr=True, compare=False)

In [48]:
p1 = Person("001", "Name-001", 12)
p2 = Person("002", "Name-002", 14)
p3 = Person("003", "Name-003", 16)
p4_not_in_list = Person("004", "Name-004", 18, False)

person_list = [p1, p2, p3]
person_list

[Person(id='001', name='Name-001', age=12, male=True),
 Person(id='002', name='Name-002', age=14, male=True),
 Person(id='003', name='Name-003', age=16, male=True)]

- DataFrame létrehozása dataclassból

In [49]:
df = pd.DataFrame.from_records([p.__dict__ for p in person_list])
df

,id,name,age,male
0,001,Name-001,12,True
1,002,Name-002,14,True
2,003,Name-003,16,True


In [50]:
df = df._append(p4_not_in_list.__dict__, ignore_index=True)
df

,id,name,age,male
0,001,Name-001,12,True
1,002,Name-002,14,True
2,003,Name-003,16,True
3,004,Name-004,18,False


- DataFrame dataclassá alakítása

In [51]:
persons = df.apply(lambda row: Person(*row), axis=1).to_list()
persons

[Person(id='001', name='Name-001', age=12, male=True),
 Person(id='002', name='Name-002', age=14, male=True),
 Person(id='003', name='Name-003', age=16, male=True),
 Person(id='004', name='Name-004', age=18, male=False)]

### DataFramek exportálása

In [52]:
df

,id,name,age,male
0,001,Name-001,12,True
1,002,Name-002,14,True
2,003,Name-003,16,True
3,004,Name-004,18,False


#### Exportálás CSV formátumba 

- Az index=False-ra azért van szükség, mert enélkül a pandas alapértelmezetten elmenti a saját index értékeit is a csv fájlba.

In [53]:
df.to_csv("persons.csv", index=False)

#### Exportálás JSON formátumba

- Az orient="records"-ra azért van szükség, mert így menti el soronként/recordonként az adatokat

In [54]:
df.to_json("persons.json", orient="records")

#### Exportálás XLSX formátumba

- openpyxl modul szükséges az exportáláshoz

In [55]:
df.to_excel("persons.xlsx", index=True)

### DataFramek importálása

In [56]:
df = pd.read_csv("persons.csv")
df

,id,name,age,male
0,1,Name-001,12,True
1,2,Name-002,14,True
2,3,Name-003,16,True
3,4,Name-004,18,False


In [57]:
persons_csv = df.apply(lambda row: Person(*row), axis=1).to_list()
persons_csv

[Person(id=1, name='Name-001', age=12, male=True),
 Person(id=2, name='Name-002', age=14, male=True),
 Person(id=3, name='Name-003', age=16, male=True),
 Person(id=4, name='Name-004', age=18, male=False)]

In [58]:
df = pd.read_json("persons.json")
df

,id,name,age,male
0,1,Name-001,12,True
1,2,Name-002,14,True
2,3,Name-003,16,True
3,4,Name-004,18,False


In [59]:
df = pd.read_excel("persons.xlsx")
df

,Unnamed: 0,id,name,age,male
0,0,1,Name-001,12,True
1,1,2,Name-002,14,True
2,2,3,Name-003,16,True
3,3,4,Name-004,18,False


### Pandas adatbázist is tud kezelni

In [60]:
import sqlite3
import pandas as pd

# SQLlite adatbázis készítése a memóriában
conn = sqlite3.connect(':memory:')

cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE persons (
        id TEXT,
        name TEXT,
        age INTEGER,
        male BOOLEAN
    )
''')

# Teszt adatok betöltése
cursor.execute("INSERT INTO persons VALUES ('001', 'Alice', 30, 0)")
cursor.execute("INSERT INTO persons VALUES ('002', 'Bob', 25, 1)")
cursor.execute("INSERT INTO persons VALUES ('003', 'Charlie', 28, 1)")

conn.commit()

# SQL parancs futtatása pandas segítségével, amely egy dataframmel tér vissza.
df = pd.read_sql('SELECT * FROM persons', conn)

df

,id,name,age,male
0,001,Alice,30,0
1,002,Bob,25,1
2,003,Charlie,28,1


In [61]:
persons = df.apply(lambda row: Person(*row), axis=1).to_list()
persons

[Person(id='001', name='Alice', age=30, male=0),
 Person(id='002', name='Bob', age=25, male=1),
 Person(id='003', name='Charlie', age=28, male=1)]

### Vizualizáció pandas segítségével

In [62]:
df

,id,name,age,male
0,001,Alice,30,0
1,002,Bob,25,1
2,003,Charlie,28,1


In [ ]:
df.plot.bar()

In [ ]:
df.plot.area(alpha=0.4)